In [49]:
import cv2
import numpy as np
import math
from random import randint
import itertools

In [50]:
from scipy.fft import fft2, fftshift

In [51]:
def FFT_(image):    
    f = fft2(image)
    # Shift the zero-frequency component to the center
    fshift = fftshift(f)
    # Compute magnitude spectrum
    magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-8)  # add epsilon to avoid log(0)
    max_abs_val = np.max(np.abs(magnitude_spectrum))
    im = np.abs(magnitude_spectrum)/max_abs_val
    return im

In [52]:
def crop_to_circle(image):
    """Crop the input image to a circular shape."""
    height, width = image.shape
    center = (width // 2, height // 2)
    radius = min(center)
    
    # Create a circular mask
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.circle(mask, center, radius, 255, -1)
    
    # Apply the mask to the image
    cropped = cv2.bitwise_and(image, image, mask=mask)
    return cropped, center, radius
def generate_nail_positions(center, radius, num_nails):
    """Generate evenly spaced nail positions around the circle."""
    nails = []
    for i in range(num_nails):
        angle = 2 * math.pi * i / num_nails
        x = int(center[0] + radius * math.cos(angle))
        y = int(center[1] + radius * math.sin(angle))
        nails.append((x, y))
    return nails
def combination(nail_positions):
    comb = np.array(list(itertools.combinations(range(len(nail_positions)),2)))
    return comb

In [60]:
def string_art_generator(image, num_nails=200, max_iterations=1000):
    canvas, center_, radius_ = crop_to_circle(np.ones((image.shape)))
    cropped_image, center, radius = crop_to_circle(image)

    nail_positions = generate_nail_positions(center, radius, num_nails)
    comb_nail = combination(nail_positions)
    FFT_dict = {}
    for i,j in comb_nail:
        im = canvas.copy()
        cv2.line(im,nail_positions[i],nail_positions[j], (0,0,0), 1)
        FFT_dict[nail_positions[i],nail_positions[j]] = FFT_(im)
    
    visited_pairs = set()
    orginal_FFT = FFT_(cropped_image)
    
    iter_ = 0
    count = 0
    # Main loop
    for _ in range(max_iterations - 1):
        iter_ += 1
        print("iteration : " , iter_)
        min_integral = float(np.inf)
        count = 0
        for x, y in comb_nail:
            canvas_fft = FFT_(canvas)
            count += 1
            if count % 50 == 0 : print("count : " , count)

            curr_n =  nail_positions[x]
            next_n = nail_positions[y]
            canvas_fft = canvas_fft + FFT_dict[(curr_n,next_n)]
            canvas_fft = canvas_fft / canvas_fft.max()

            diff_ = np.mean(np.abs(orginal_FFT.flatten() - canvas_fft.flatten()))
            if diff_ < min_integral:  
                min_integral = diff_
                a,b = curr_n , next_n
          
        # If no valid next nail is found, break
        if min_integral == np.inf:
            print("broken")
            break
        canvas = cv2.line(canvas, a, b, (0,0,0), 1) 
        
    cv2.imshow('String Art',canvas)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    return canvas


In [58]:
image = cv2.imread('f1.jpg', cv2.IMREAD_GRAYSCALE)
image = cv2.resize(image, (900,900))
image = image / image.max()
#image = cv2.bitwise_not(image)
#_, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)

cv2.imshow('String Art', image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [63]:
string_art = string_art_generator(image, num_nails=100, max_iterations=100)

KeyboardInterrupt: 